# Domain elements and `MetaTerm`s

The lambda notebook metalanguage supports a notion of "ontological reference" via expressions of class **`lamb.meta.meta.MetaTerm`**. Essentially, any type in the type system can correspond to some set of **domain elements**, and these domain elements are potential objects of reference for expressions of that type. A `MetaTerm` instantiates a reference to a domain element directly into the metalanguage. Domain elements are in general modeled via arbitrary python objects that are "outside" of the metalanguage itself. In certain cases, there are natural candidates in python that have similar semantics (relative to the metalanguage, metametasemantics) to what we are looking for as the object of reference. As a simple example, the following cell produces an expression consisting only of a `MetaTerm` at type `t`:

In [ ]:
%te True

`MetaTerm`s instantiated via the parser accept a type annotation with concrete types (no polymorphism), but this is never required; a mismatch will produce an error. All parsable `MetaTerm`s have a type that is inferrable entirely from their description.

In [ ]:
%te True_e

**Identifying `MetaTerm`s** (preliminaries): in rich reprs `MetaTerm`s are rendered with a sans serif font, and in plain reprs, a prefixing `_` symbol indicates meta status (`True`/`False` and numbers excluded). In code, an expression's meta status can be programmatically checked via the `TypedExpr` member function `meta()`; this is better than doing an explicit `isinstance` check.

**`MetaTerms` vs. terms in general**:
In general, `MetaTerm`s can be thought of as a special kind of term that wraps the domain element and allows it to appear in the metalanguage; they compare equal (and hash equal) to their domain element counterpart. They are implemented as terms (and satisfy `TypedExpr.term()`) but are never counted as free terms, cannot be changed by assignments, cannot be bound, etc.

Though they are both terms, `TypedTerm` constants are not `MetaTerm`s! This may be a somewhat weird conceptual distinction depending on how you were taught logic. The intuition is that a `MetaTerm` is reference to the *actual thing*, whereas a regular constant is more like a name for some thing. (Of course, in computational modeling we never really have the actual thing; in this case just a python object that is yet another proxy.) In the metalanguage, regular constants are not automatically valued to domain elements without being filled in by an assignment (or execution context). In the lambda notebook metalanguage, a regular constant is just a free term that cannot be bound within a metalanguage expression, and a variable is a term that may be bound (or may not be); `MetaTerm`s are not free. Another way to put it: a regular constant is something whose value will be a single domain element, but without an assignment we don't know what that value is, and it could be anything in the domain. A `MetaTerm` can only have the value of the thing to which it refers, and this is entirely assignment-independent.

## Default basic type domains
### Booleans

A simple starting point is type `t`. This type's domain is quite straighforwardly a set consisting of (only) python `True` and `False`. The corresponding `MetaTerm`s are instantiated in the parser in a similar way to how you'd do it in python. The following expression produces a `TypedExpr` set consisting of the `MetaTerm`s `True` and `False`:

In [ ]:
%te {True, False}

As you'd expect, the terms compare equal to the python values:

In [ ]:
meta.MetaTerm(True) == True

In [ ]:
meta.MetaTerm(False) == False

In the metalanguage, as in python, `True` and `False` are reserved terms that can only instantiate `MetaTerm`s; while you can write constants at this type, they can't be named as `True` or `False`.

Type domains can be accessed in python via the `domain` member variable, and supports various things, including membership checks. Finite domains support `len` and (if marked as `enumerable`) iteration as well.

In [ ]:
tp("t").domain.cardinality()

In [ ]:
list(tp("t").domain)

Anything in a domain element is safe to instantiate as a `MetaTerm`.

In [ ]:
[meta.MetaTerm(e) for e in tp("t").domain]

In [ ]:
True in tp("t").domain

In [ ]:
meta.MetaTerm(True) in tp("t").domain

In [ ]:
1 in tp("t").domain

**Programming caveat**. If you are an expert python programmer, one caveat to be aware of is that `MetaTerm`s are strictly typed, with everything that comes with that. In python, there's a set of `falsy` elements which act like `False` for boolean purposes and in some cases even compare equal. `MetaTerm(False)` certainly acts falsy when used in python, but it will not compare equal to any of these elements, and these elements will not act as part of the domain.

For related reasons, when working with metalanguage objects in python, it's very important to differentiate checks for python `None` (use: `x is None`) from bool checks (use the verbose `te == False` or `te == True` to avoid accidental equation of `None` returns with `False` or other truthy/falsy elements that are not domain elements for type `t`.

In [ ]:
# demonstration of falsiness. Note that `and` will return the left element for this case.
meta.MetaTerm(False) and False

In [ ]:
False == 0 # these compare as equal

In [ ]:
meta.MetaTerm(False) == 0 # these don't

A caveat to the caveat. For various reasons, when writing metalanguage expressions explicitly annotating numbers and booleans with types `t` and `n` respectively is allowed and will parse by the expected conversion according to the type, rather than erroring.

In [ ]:
%te 0_t

In [ ]:
te("0_t") == 0

### Numbers

The metalanguage supports python integers via type `n`. These are pretty straightforward as well, and their metalanguage and python semantics are identical for the standard operations.

In [ ]:
%te 10

In [ ]:
meta.MetaTerm(10) == 10

In [ ]:
5 in tp("n").domain

The main immediate difference to booleans is that the domain set for this type is not modeled as finite!

In [ ]:
# len will raise a ValueError
tp("n").domain.cardinality()

Iteration is supported, but is for obvious reasons not terminating. It will iterate over ints only, from 0 alternating positive and negative.

In [ ]:
it = iter(tp("n").domain)
for i in range(10):
    print(next(it))

Parsing note: the metalanguage supports a unary `-` operator, but negative integers will be parsed directly to the negative value, rather than a complex expression with unary negation.

Limitation: The `MetaTerm` class will take any python numeric, though parsed metalanguage expressions currently support only integers.

In [ ]:
(meta.MetaTerm(2.5) - 1.25).simplify_all().derivation

### Entities

The domain `e` (entities) instantiates a third standard pattern for domain elements. In contrast to the previous two cases, entities are not modeled using python objects that have any relevant semantics to the logical notion of an entity. Rather, they are modeled as arbitrary elements that have no meta-meta-relationship to each other beyond distinguishability.

* Of course, it would be possible to implement a richer meta-meta system of entities, e.g. with mereology! But this is not the default setup. Here we build on standard first order model theory.

At a technical level, entities are modeled on the python side as strings with a prefix `_c` (mnemonic: "constant") followed by a non-negative integer. In various context the underscore may be optional, and it is primarily a parser signal to construct a `MetaTerm`. It may be used with any meta-element including numbers and booleans, but it is required for str-backed domains. It doesn't appear in rich reprs (sans serif is used instead), but it does appear in ordinary reprs.

In [ ]:
%te _c5

In [ ]:
repr(te("_c5"))

Domains like this inherit python equality straightforwardly.

In [ ]:
%te simplify _c5 <=> _c6
_llast.derivation

In [ ]:
%te simplify _c5 <=> _c5
_llast.derivation

As a cautionary note, something like `c5_e` will parse as a regular variable, not a MetaTerm:

In [ ]:
%te simplify c5_e <=> _c5

This example raises a more general point: what is the relationship between variables or regular free terms at some type and `MetaTerm`s at the same type? The short answer is that the free term can be thought of as a slot that could be filled in by a type domain element. An equality statement like the above cannot be fully resolved without knowing what `c_e` stands in for; this goes whether it's a constant or a variable. Of course, if `c_e` is bound rather than free, then its binder may determine values in some way.

In [ ]:
%te simplify (L x_e : x <=> _c5)(_c3)
_llast.derivation

Like numbers, string-backed domains are not treated as finite, and support non-terminating iteration. Since the values essentially correspond to natural numbers (including 0), iteration proceeds in a standard order for natural numbers.

In [ ]:
it = iter(tp("e").domain)
for i in range(10):
    print(next(it))

### String-backed domains more generally

Entities instantiate the general concept of a "string-backed domain", where domain elements are modeled as some arbitrary prefix followed by a sequence of digits. When you add a new atomic type, typically you will want a corresponding string-backed domain set. This is set up by default.  It is required that distinct string-backed domains be associated with a unique prefix that allows completely distinguish elements in one domain from another.

By default, the prefix for entities will be `c`, for type `v` (conventionally, events) will be `e`, and for type `s` (conventionally, intensions) will be `w`, but these can all be overridden.

In [ ]:
lang.get_system().add_basic_type(types.BasicType("s"))

In [ ]:
%te _w1

In [ ]:
it = iter(tp("s").domain)
for i in range(10):
    print(next(it))

## Derived/complex type domains

Complex concrete type constructors (`SetType`, `TupleType`, `FunType`) also have corresponding type domains. This support membership checking, iteration, etc, and inherit cardinality from their component types. A general caveat is that often, even in the finite case, the sets you can generate this way are quite large!

In general, the metalanguage parser doesn't support instantiating `MetaTerm`s for these domains directly. However, they can be constructed directly via the `MetaTerm` constructor, and are used internally for certain calculations. Set and tuple domain elements have exact analogues using `ListedSet` typed expressions and `Tuple` typed expressions that the parser will construct, which compare as expected, etc.

### `SetType` domains

Perhaps unsurprisingly, these are backed by python `collections.abc.Set` objects, in particular `frozenset`s. (This is for technical reasons: because regular python `set` is mutable, it does not support sets of sets.) For example, here are the elements for domains `{t}` and `{{t}}`:

In [ ]:
for e in tp("{t}").domain:
    print(e)

In [ ]:
for e in tp("{{t}}").domain:
    print(e)

### `TupleType` domain elements

Tuples are also pretty straightforwardly backed by python `tuple`s. Here are a few examples of iteration over finite tuple domains:

In [ ]:
for e in tp("(t,t)").domain:
    print(e)

In [ ]:
for e in tp("(t,(t,t))").domain:
    print(e)

### `FunType` domain elements

Python backing for functional elements is a bit more complicated than the previous two cases. There are three possible backing objects:

1. Python `set`s (really, `frozenset`s): a set in this context gives the function that characterizes that set, relative to the type domain. That is, the function returns true iff a type domain element is a member of that set.
2. Python `dict` objects (see details on type below): an explicit mapping of domain elements to the image of the function.
3. An arbitrary python `callable`.

Each of these has some complications and caveats that this document will only touch on. This type domain supports iteration, but the iterator will only produce `dict`s. In fact, from the examples below you can see that this domain does not exactly use `dict`s, but rather objects of class `lamb.utils.frozendict`. This class provides a minimal implementation of a immutable (and therefore hashable) `collections.abc.Mapping` class that is suitable for recursion.

In [ ]:
for e in tp("<t,t>").domain:
    print(e)

In [ ]:
for e in tp("<(t,t),t>").domain:
    print(e)

**Set-backed functions**. A set-backed functional term can be constructed directly by supplying an appropriate type together with a set to the `MetaTerm` constructor. (Without this type, a set will be construed as a `SetType`.) As with other cases, this normalizes, so you don't have to start with a `frozenset`.

In [ ]:
meta.MetaTerm({True}, typ=tp("<t,t>"))

Reduction is simply implemented as set membership checking for this case:

In [ ]:
meta.MetaTerm({True}, typ=tp("<t,t>"))(False).reduce_all().derivation

**Mapping-backed functions**. These cases at first seem straightforward, and they are, as long as the domain of the mapping covers the type domain:

In [ ]:
meta.MetaTerm({False: False, True: False})

In [ ]:
meta.MetaTerm({False: False, True: True})(False).reduce_all().derivation

The complication comes in when the mapping is partial relative to the type domain. The default behavior is to raise an exception of type `meta.meta.OutOfDomain` for this case.

In [ ]:
with lamb.errors():
    meta.MetaTerm({True: True})(False).reduce_all().derivation

This behavior can be relaxed by explicitly invoking a non-reduce simplify pass with `strict_charfuns=False`. With this simplify option, a type domain element missing from the `dict` yields a `False` return value.

In [ ]:
meta.MetaTerm({True: True})(False).simplify_all(reduce=False, strict_charfuns=False).derivation

Mapping-backed functions can be defined directly in the metalanguge by using python `dict` notation. (The one exception is that the empty function cannot be defined this way, because `{}` refers to the empty set in the metalanguage.)

In [ ]:
%te {False: False, True: False}

In [ ]:
%te {False: False, True: False}(True)

In [ ]:
%te reduce {False: False, True: False}(True)

**`callable`-backed functions**. These primarily exist for internal use, and are quite unconstrained, with no real type-safety -- nothing about the API enforces that the type you provide matches the behavior of the function you provide. Best to only use these if you really know what you're doing. A type must be explicitly provided.

In [ ]:
f = meta.MetaTerm((lambda x: x), typ=tp("<X,X>"))
f

In [ ]:
f(te("_c1")).reduce_all()

**More caveats**: Functional type domain elements don't have an identity criteria in the way that other type domains do. This is obviously worst for the `callable` backed cases, but comparison across `set` and `dict`-backed cases doesn't work.

## Domain restriction

The documentation related to type domains and corresponding concepts like quantification is littered with caveats like, "only safe for finite domains". Most semantic applications do not assume that domains like `e` and `s` are finite. However, it is common practice to demonstrate analyses on "toy" subdomains. The type domain system provides tools for doing this for atomic types (as do other aspects of the evaluation system not covered in this document).

As an example, here's a simple set expression that by default, we cannot convert to a `ListedSet`, because the type domain for `e` is not assumed to be finite, and therefore an `eliminate` call does nothing:

In [ ]:
te("Set x_e : True").eliminate()

However, with a toy domain, we could eiminate this set expression. There are two basic ways to do this. First, and safest, is to use the `restrict_domain` context manager on a type. This affects the basic type and all corresponding derived types. Here are a few examples for picking a small part of type `e`:

In [ ]:
with types.type_e.restrict_domain(values=['_c0', '_c1', '_c2']):
    display(te("Set x_e : True").eliminate())
    display(te("Set x_{e} : True").eliminate())
    display(te("Set x_<e,t> : True").eliminate())

This context manager function also takes a parameter `count=n`, which when set, just gives you the first `n` type domain elements by iteration order.

Second, it is possible to do more long-lasting domain restrictions just by setting the domain directly:

In [ ]:
type_e = tp("e")
type_e.domain = type_e.get_subdomain(count=3)
type_e.domain

In [ ]:
te("Set x_e : True").eliminate()

Trying to instantiate `MetaTerm`s that are not possible in the current subdomain but would be valid in a superdomain will produce an error:

In [ ]:
%te _c3

To reset a domain restriction like this, you can get at the original domain via the `superdomain` member variable. You could also just call `meta.reset_type_system()`, though this will of course reset all changes you might have made, not just this one.

In [ ]:
tp("e").domain = tp("e").domain.superdomain
tp("e").domain

Both of these methods of domain restriction support repeated restriction calls; `superdomain` will store the previous one.

## `MetaTerm` simplification and execution

These objects have very straightforward simplification and execution behavior. They don't simplify -- they are atomic -- and their compiled instances just give back the contained domain elements independent of context.

* Calling `simplify` on a `MetaTerm` does do exactly one thing: it rechecks the domain element against any current type domain restrictions. A failure of this check raises a `DomainError`.

Things get a bit more complex in terms of their behavior when embedded in complex expressions, but generally things work exactly as you'd expect for both simplification and execution.

* When dealing with complex data structures like sets and tuples in python code, there are normalization checks and functions `meta.core.is_concrete` and `meta.core.to_concrete` that let you consistently compare across ways these structures could be represented at the same type (for example, a `ListedSet` of `MetaTerm`s vs. a `MetaTerm` set of domain elements). See the set documentation for more information. This is automatically taken care of in metalanguage expressions.
* Functions were discussed above, where application works but has some caveats depending on the backing python type chosen. Compiled functions at concrete types can be safely rewrapped in `MetaTerm`s.